In [18]:
import os
import pandas as pd
from cedalion.io import bids
import snirf2bids as s2b
from pathlib import Path
import re

## Convert a fNIRS dataset to BIDS

This notebook automates the conversion of an fNIRS dataset into a BIDS-compliant format. 
To begin, you'll need to:

1. Dataset Path: The folder containing the raw dataset.
2. Destination Path: The folder where the BIDS-compliant files will be saved.
3. Mapping CSV File: A CSV file that defines the dataset folder structure and includes the necessary details for constructing the BIDS structure.
4. Optional Metadata: Any additional metadata you want to include. You can use [this google form](https://docs.google.com/forms/d/e/1FAIpQLSeZjlgIqCwp054HsHmTBKPziqcOlfTcaWpdXcGFYPDf0Q5vNg/viewform?usp=sf_link) or [this website](https://neurojson.org/Create/dataset_description_fnirs) to create this json file.

In [19]:
dataset_path = 'path-to-your-dataset'
destination_path = 'your-destination-bids-path'

dataset_path = Path(dataset_path)
destination_path = Path(destination_path)

In [20]:
mapping_df_path = bids.get_snirf2bids_mapping_csv(dataset_path)
extra_meta_data_path = Path("path-to-you-meta-data")

extra_meta_data_path = extra_meta_data_path if os.path.exists(extra_meta_data_path) else None

`get_snirf2bids_mapping_csv` helps you create your mapping CSV file. After generating the CSV file, you might need to manually edit it to include additional information or make adjustments as required.

A valid mapping CSV must include all SNIRF files in your dataset, along with the following details for each file:

- sub: The identifier for the participant.
- ses (optional): The session identifier, if applicable.
- task: The task name or label.
- run (optional): The run number, if applicable.
- acq (optional): The acquisition number, if applicable.
- cond (optional): Conditions' keys as a list.
- cond_match (optional): Conditions' values as a list.
- duration (optional): Events' duration. 

In [21]:
mapping_df = pd.read_csv(mapping_df_path, dtype=str)
mapping_df.head(10)

,current_name,sub,ses,task,run,acq,cond,cond_match,duration
0,sub-13/ses-01/nirs/sub-13_ses-01_task-WordStro...,13,01,WordStroop,01,NaN,NaN,NaN,NaN
1,sub-14/ses-01/nirs/sub-14_ses-01_task-WordStro...,14,01,WordStroop,01,NaN,NaN,NaN,NaN
2,sub-22/ses-01/nirs/sub-22_ses-01_task-WordStro...,22,01,WordStroop,01,NaN,NaN,NaN,NaN
3,sub-25/ses-01/nirs/sub-25_ses-01_task-WordStro...,25,01,WordStroop,01,NaN,NaN,NaN,NaN
4,sub-24/ses-01/nirs/sub-24_ses-01_task-WordStro...,24,01,WordStroop,01,NaN,NaN,NaN,NaN
5,sub-23/ses-01/nirs/sub-23_ses-01_task-WordStro...,23,01,WordStroop,01,NaN,NaN,NaN,NaN
6,sub-15/ses-01/nirs/sub-15_ses-01_task-WordStro...,15,01,WordStroop,01,NaN,NaN,NaN,NaN
7,sub-12/ses-01/nirs/sub-12_ses-01_task-WordStro...,12,01,WordStroop,01,NaN,NaN,NaN,NaN
8,sub-08/ses-01/nirs/sub-08_ses-01_task-WordStro...,08,01,WordStroop,01,NaN,NaN,NaN,NaN
9,sub-01/ses-02/nirs/sub-01_ses-02_task-WordStro...,01,02,WordStroop,01,NaN,NaN,NaN,NaN


The mapping table serves as a key component for organizing and processing your dataset. 

The `ses`, `run`, and `acq` columns are optional and can be set to None if not applicable. The `current_name` column contains the path to the SNIRF files in your dataset.

### Looking for possible *_scan.tsv files

To ensure no important information (e.g., acquisition time) from the original dataset is lost, we will:

- Search Subdirectories: Traverse through all subdirectories within the dataset.
- Locate Existing Scan Files: Search for all *_scan.tsv files in the dataset.
- Integrate into Mapping Table: Extract the relevant information from these files and add it to our mapping table.
- Extracts acquisition time from SNIRF files if missing in the `_scans.tsv` file.

This approach ensures that any details, such as acquisition time, are retained and incorporated into the BIDS-compliant structure.

In [22]:
mapping_df["filename_org"] = mapping_df["current_name"].apply(
    lambda x: os.path.basename(x))
scan_df = bids.search_for_acq_time_in_scan_files(dataset_path)

mapping_df = pd.merge(mapping_df, scan_df, on="filename_org", how="left")
mapping_df["acq_time"] = mapping_df.apply(bids.search_for_acq_time_in_snirf_files, axis=1, args=(dataset_path,))

mapping_df.head(10)

,current_name,sub,ses,task,run,acq,cond,cond_match,duration,filename_org,acq_time
0,sub-13/ses-01/nirs/sub-13_ses-01_task-WordStro...,13,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-13_ses-01_task-WordStroop_run-01_nirs,2023-02-24T17:40:50
1,sub-14/ses-01/nirs/sub-14_ses-01_task-WordStro...,14,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-14_ses-01_task-WordStroop_run-01_nirs,2023-03-06T11:30:09
2,sub-22/ses-01/nirs/sub-22_ses-01_task-WordStro...,22,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-22_ses-01_task-WordStroop_run-01_nirs,2024-02-02T15:20:21
3,sub-25/ses-01/nirs/sub-25_ses-01_task-WordStro...,25,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-25_ses-01_task-WordStroop_run-01_nirs,2024-02-05T17:10:51
4,sub-24/ses-01/nirs/sub-24_ses-01_task-WordStro...,24,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-24_ses-01_task-WordStroop_run-01_nirs,2024-02-05T13:23:52
5,sub-23/ses-01/nirs/sub-23_ses-01_task-WordStro...,23,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-23_ses-01_task-WordStroop_run-01_nirs,2024-02-04T17:47:18
6,sub-15/ses-01/nirs/sub-15_ses-01_task-WordStro...,15,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-15_ses-01_task-WordStroop_run-01_nirs,2023-03-06T17:10:46
7,sub-12/ses-01/nirs/sub-12_ses-01_task-WordStro...,12,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-12_ses-01_task-WordStroop_run-01_nirs,2023-02-24T11:19:51
8,sub-08/ses-01/nirs/sub-08_ses-01_task-WordStro...,08,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-08_ses-01_task-WordStroop_run-01_nirs,2023-01-09T10:51:55
9,sub-01/ses-02/nirs/sub-01_ses-02_task-WordStro...,01,02,WordStroop,01,NaN,NaN,NaN,NaN,sub-01_ses-02_task-WordStroop_run-01_nirs,2022-12-11T19:23:51


The `acq_time` information is retrieved from the original dataset's *_scan.tsv files and integrated into the mapping table.

### Looking for possible *_session.tsv files

Similar to *_scan.tsv files, we search for *_session.tsv files in the dataset path to capture additional session-level metadata, such as acquisition times. Any relevant information from these files is added to the mapping table to ensure all session details are preserved.

In [23]:
session_df = bids.search_for_sessions_acq_time(dataset_path)
mapping_df = pd.merge(mapping_df, session_df, on=["sub", "ses"], how="left")

mapping_df.head(10)

,current_name,sub,ses,task,run,acq,cond,cond_match,duration,filename_org,acq_time,ses_acq_time
0,sub-13/ses-01/nirs/sub-13_ses-01_task-WordStro...,13,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-13_ses-01_task-WordStroop_run-01_nirs,2023-02-24T17:40:50,None
1,sub-14/ses-01/nirs/sub-14_ses-01_task-WordStro...,14,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-14_ses-01_task-WordStroop_run-01_nirs,2023-03-06T11:30:09,None
2,sub-22/ses-01/nirs/sub-22_ses-01_task-WordStro...,22,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-22_ses-01_task-WordStroop_run-01_nirs,2024-02-02T15:20:21,None
3,sub-25/ses-01/nirs/sub-25_ses-01_task-WordStro...,25,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-25_ses-01_task-WordStroop_run-01_nirs,2024-02-05T17:10:51,None
4,sub-24/ses-01/nirs/sub-24_ses-01_task-WordStro...,24,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-24_ses-01_task-WordStroop_run-01_nirs,2024-02-05T13:23:52,None
5,sub-23/ses-01/nirs/sub-23_ses-01_task-WordStro...,23,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-23_ses-01_task-WordStroop_run-01_nirs,2024-02-04T17:47:18,None
6,sub-15/ses-01/nirs/sub-15_ses-01_task-WordStro...,15,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-15_ses-01_task-WordStroop_run-01_nirs,2023-03-06T17:10:46,None
7,sub-12/ses-01/nirs/sub-12_ses-01_task-WordStro...,12,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-12_ses-01_task-WordStroop_run-01_nirs,2023-02-24T11:19:51,None
8,sub-08/ses-01/nirs/sub-08_ses-01_task-WordStro...,08,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-08_ses-01_task-WordStroop_run-01_nirs,2023-01-09T10:51:55,None
9,sub-01/ses-02/nirs/sub-01_ses-02_task-WordStro...,01,02,WordStroop,01,NaN,NaN,NaN,NaN,sub-01_ses-02_task-WordStroop_run-01_nirs,2022-12-11T19:23:51,None


### Create BIDS Folder Structure

The goal of this section is to rename the SNIRF files according to the BIDS naming convention and place them in the appropriate directory under `destination_path`, following the BIDS folder structure.

Steps:
1. Generate New Filenames: Create BIDS-compliant filenames for all SNIRF records.
2. Determine File Locations: Identify the appropriate locations for these files within the BIDS folder hierarchy.

This process ensures that the dataset adheres to BIDS standards for organization and naming.

In [24]:
mapping_df[["bids_name", "parent_path"]] = mapping_df.apply(
    bids.create_bids_standard_filenames, axis=1, result_type='expand')

mapping_df.head(10)

,current_name,sub,ses,task,run,acq,cond,cond_match,duration,filename_org,acq_time,ses_acq_time,bids_name,parent_path
0,sub-13/ses-01/nirs/sub-13_ses-01_task-WordStro...,13,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-13_ses-01_task-WordStroop_run-01_nirs,2023-02-24T17:40:50,None,sub-13_ses-01_task-WordStroop_run-01_nirs.snirf,sub-13/ses-01/nirs
1,sub-14/ses-01/nirs/sub-14_ses-01_task-WordStro...,14,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-14_ses-01_task-WordStroop_run-01_nirs,2023-03-06T11:30:09,None,sub-14_ses-01_task-WordStroop_run-01_nirs.snirf,sub-14/ses-01/nirs
2,sub-22/ses-01/nirs/sub-22_ses-01_task-WordStro...,22,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-22_ses-01_task-WordStroop_run-01_nirs,2024-02-02T15:20:21,None,sub-22_ses-01_task-WordStroop_run-01_nirs.snirf,sub-22/ses-01/nirs
3,sub-25/ses-01/nirs/sub-25_ses-01_task-WordStro...,25,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-25_ses-01_task-WordStroop_run-01_nirs,2024-02-05T17:10:51,None,sub-25_ses-01_task-WordStroop_run-01_nirs.snirf,sub-25/ses-01/nirs
4,sub-24/ses-01/nirs/sub-24_ses-01_task-WordStro...,24,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-24_ses-01_task-WordStroop_run-01_nirs,2024-02-05T13:23:52,None,sub-24_ses-01_task-WordStroop_run-01_nirs.snirf,sub-24/ses-01/nirs
5,sub-23/ses-01/nirs/sub-23_ses-01_task-WordStro...,23,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-23_ses-01_task-WordStroop_run-01_nirs,2024-02-04T17:47:18,None,sub-23_ses-01_task-WordStroop_run-01_nirs.snirf,sub-23/ses-01/nirs
6,sub-15/ses-01/nirs/sub-15_ses-01_task-WordStro...,15,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-15_ses-01_task-WordStroop_run-01_nirs,2023-03-06T17:10:46,None,sub-15_ses-01_task-WordStroop_run-01_nirs.snirf,sub-15/ses-01/nirs
7,sub-12/ses-01/nirs/sub-12_ses-01_task-WordStro...,12,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-12_ses-01_task-WordStroop_run-01_nirs,2023-02-24T11:19:51,None,sub-12_ses-01_task-WordStroop_run-01_nirs.snirf,sub-12/ses-01/nirs
8,sub-08/ses-01/nirs/sub-08_ses-01_task-WordStro...,08,01,WordStroop,01,NaN,NaN,NaN,NaN,sub-08_ses-01_task-WordStroop_run-01_nirs,2023-01-09T10:51:55,None,sub-08_ses-01_task-WordStroop_run-01_nirs.snirf,sub-08/ses-01/nirs
9,sub-01/ses-02/nirs/sub-01_ses-02_task-WordStro...,01,02,WordStroop,01,NaN,NaN,NaN,NaN,sub-01_ses-02_task-WordStroop_run-01_nirs,2022-12-11T19:23:51,None,sub-01_ses-02_task-WordStroop_run-01_nirs.snirf,sub-01/ses-02/nirs


To facilitate proper organization:

- `parent_path`: Added to the mapping dataframe to define the location of each SNIRF file within `destination_path`.
- `bids_name`: Specifies the new BIDS-compliant name for each file.
In the following sections, we will rename all files to their corresponding `bids_name` and copy them to their designated parent_path.

In [25]:
_ = mapping_df.apply(bids.copy_rename_snirf, axis=1, args=(dataset_path, destination_path))

### Create BIDS specific files (e.g., _coordsystem.json)

In this step, we utilize the snirf2bids Python package to generate the necessary .tsv and .json files for the BIDS structure.

For every record, the following files will be created:
1. _coordsystem.json
2. _optodes.json
3. _optodes.tsv
4. *_channels.tsv
5. *_events.json
6. *_events.tsv
7. *_nirs.json

These files are essential for ensuring the dataset adheres to BIDS standards.

In [26]:
s2b.snirf2bids_recurse(destination_path)
pattern = re.compile(r'.*_scans\.tsv$|^participants\.tsv$|^temp_participants\.tsv$')
files_to_delete = [file for file in destination_path.rglob('*') if file.is_file() and pattern.match(file.name)]
for file in files_to_delete:
    file.unlink()

### Create _scan.tsv Files

Now, we proceed to create scan files for all subjects and sessions. Previously, we searched the original dataset path for any provided scan information, which will now be incorporated into the BIDS structure.

In [27]:
scan_df = mapping_df[["sub", "ses", "bids_name", "acq_time"]]
scan_df['ses'].fillna("Unknown", inplace=True)
scan_df = scan_df.groupby(["sub", "ses"])
scan_df.apply(lambda group: bids.create_scan_files(group, destination_path))

/var/folders/82/14s7dkbx0ys8_nfvyj9009180000gn/T/ipykernel_1877/2138123139.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scan_df['ses'].fillna("Unknown", inplace=True)
/var/folders/82/14s7dkbx0ys8_nfvyj9009180000gn/T/ipykernel_1877/2138123139.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  scan_df.apply(lambda group: bids.create_scan_files(group, destination_path))


""


### Create _session.tsv Files

The next step is to create session files for all subjects. As with the scan files, we previously searched the original dataset path for any session information, which will now be used to create the corresponding BIDS session files.

In [28]:
session_df = mapping_df[["sub", "ses", "ses_acq_time"]]
session_df = session_df.groupby(["sub"])
session_df.apply(lambda group: bids.create_session_files(group, destination_path))

/var/folders/82/14s7dkbx0ys8_nfvyj9009180000gn/T/ipykernel_1877/3137108340.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  session_df.apply(lambda group: bids.create_session_files(group, destination_path))


""


### Create participants tsv and json files

In this step, we gather all available participant information from the original dataset. If any participant details are provided, they will be incorporated into the BIDS structure.

Additionally, we create a template for the participants.json file with predefined columns, including:

- species
- age
- sex
- handedness

Each of these fields will include descriptive templates to ensure consistency in the BIDS-compliant structure.

In [29]:
bids.create_participants_tsv(dataset_path, destination_path, mapping_df)
bids.create_participants_json(dataset_path, destination_path)

### Create data description file

To create the dataset_description.json file, we follow these steps:

1. Search for an existing dataset_description.json in the dataset path and retain the provided information.
2. If extra_meta_data_path is specified, add the additional metadata about the dataset.
3. If neither dataset_description.json nor extra metadata is provided, use the basename of the dataset directory as the dataset name and set the BIDS version to '1.10.0'.

In [30]:
bids.create_data_description(dataset_path, destination_path, extra_meta_data_path)

### Check _coordsystem.json file

Since an empty string is not allowed for the `NIRSCoordinateSystem` key in the *_coordsystem.json file, we will populate it with "Other" to ensure BIDS compliance.

In [31]:
bids.check_coord_files(destination_path)

### Edit *_events.tsv

To allow editing of the `duration` or `trial_type` columns in the *_events.tsv files, the mapping CSV file must include the following extra columns:

1. `duration`: Specifies the new duration for each SNIRF file that needs editing.
2. cond and cond_match:
    - `cond`: A list of keys e.g. [1, 2].
    - `cond_match`: A list of corresponding values e.g. ["con", "inc"]. 
    
    These two columns will be used to create a dictionary that maps the trial_type column.

In [32]:
_ = mapping_df.apply(bids.edit_events, axis=1, args=(destination_path,))

### Creating sourcedata Directory

Finally there is this possiblity to keep your original data under sourcedata directory at your `destination_path`.

In [33]:
bids.save_source(dataset_path, destination_path)